<a href="https://colab.research.google.com/github/cabbi120101/final_multi/blob/main/Try/%EB%94%B0%EB%9D%BC%ED%95%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

     |████████████████████████████████| 127.9 MB 1.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install gensim

     |████████████████████████████████| 24.0 MB 3.2 MB/s eta 0:00:01             | 9.6 MB 5.9 MB/s eta 0:00:03
     |████████████████████████████████| 58 kB 12.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install tensorboard

     |████████████████████████████████| 5.6 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 11.1 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 152 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.0 MB 987 kB/s eta 0:00:01
     |████████████████████████████████| 97 kB 6.7 MB/s eta 0:00:011
     |████████████████████████████████| 155 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 8.0 MB/s eta 0:00:011
     |████████████████████████████████| 146 kB 8.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

from argparse import ArgumentParser

def get_args():
    parser = ArgumentParser(description="LoL outcome predict")
    parser.add_argument('--val_set_size', type=int, default=1000)

    #in-game setting
    parser.add_argument('--champ_num', type=int)
    parser.add_argument('--patch_ver', type=str, default='9.19')

    #train data
    parser.add_argument('--dropout', type=float, default=0.5)
    parser.add_argument('--lr', type=float, default=0.001)
    parser.add_argument('--weight_decay', type=float, default=0.0001)
    parser.add_argument('--uncertainty', choices=['None', 'Model', 'Data', 'Both'], default='None')
    parser.add_argument('--in_game', type=bool, default=False)
    parser.add_argument('--batch_size', type=int, default=32)
    parser.add_argument('--epochs', type=int, default=15)
    parser.add_argument('--mc_samples', type=int, default=20)
    parser.add_argument('--use_cuda', type=bool, default=False)

    parser.add_argument('--embed_type', choices=['one_hot', 'champ2vec'], default='champ2vec')

    #tensorboard
    parser.add_argument('--log_path', type=str, default='./logs/')

    #model_save
    parser.add_argument('--model_save_path', type=str, default='./save_models/')
    parser.add_argument('--saved_model_time', type=str, default=None)
    
    args = parser.parse_args()

    return args

In [ ]:
import torch
from torch.utils.data import Dataset
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import pickle
import pdb

class Preprocess(object):
    def lolDataSet(self, arg, sep):

        path='./LOLData/'
        if arg.in_game:
            match = pd.read_csv(path+"inGameMatchData"+arg.patch_ver+sep+".csv")
        else:
            match = pd.read_csv(path+"preMatchData"+arg.patch_ver+sep+".csv")
        match.head()

        teams = ["blue", "red"]
        positions = ["_1", "_2", "_3", "_4", "_5"]
        champ_cols = []

        all_champs = set()
        for t in teams:
            for p in positions:
                col = t + p
                all_champs |= set(match[col].unique())
                champ_cols.append(col)
        allChamp = list(all_champs)

        blueChampComp = match.loc[:,['blue_1','blue_2','blue_3','blue_4','blue_5']].values
        redChampComp = match.loc[:,['red_1','red_2','red_3','red_4','red_5']].values
        if arg.in_game:
            inGameData = match.loc[:,['blue_death_diff','red_death_diff','gold_diff','xp_diff', 'timeline']].values.astype(np.int32)
        champComp = np.concatenate([blueChampComp,redChampComp]).tolist()
        matchComp = np.concatenate([blueChampComp,redChampComp], axis = -1)
        blueWin = match.loc[:,['blue_win']].values

        champ2num = {champ:i for i, champ in enumerate(allChamp)}

        for i in range(len(blueChampComp)):
            for j in range(len(blueChampComp[i])):
                blueChampComp[i][j] = champ2num[blueChampComp[i][j]]
        for i in range(len(redChampComp)):
            for j in range(len(redChampComp[i])):
                redChampComp[i][j] = champ2num[redChampComp[i][j]]

        blueChampComp = blueChampComp.astype(np.int32)
        redChampComp = redChampComp.astype(np.int32)
        blueOneHot, redOneHot = np.zeros((len(blueChampComp),len(allChamp))).astype(int), np.zeros((len(redChampComp),len(allChamp))).astype(int)
        for i in range(len(blueChampComp)):
            for j in range(len(blueChampComp[i])):
                blueOneHot[i][blueChampComp[i][j]] = 1
        for i in range(len(redChampComp)):
            for j in range(len(redChampComp[i])):
                redOneHot[i][redChampComp[i][j]] = 1
        
        if arg.in_game:
            matchComp = np.concatenate([blueOneHot,redOneHot,inGameData], axis = -1)
        else:
            matchComp = np.concatenate([blueOneHot,redOneHot], axis = -1)

        return allChamp, matchComp, blueWin

class ChampDataset(Dataset):
    def __init__(self, train_x, train_y):
        self.len = train_x.shape[0]
        self.train_x = torch.from_numpy(train_x)
        self.train_y = torch.from_numpy(train_y)

    def __getitem__(self, index):
        return self.train_x[index], self.train_y[index]

    def __len__(self):
        return self.len

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.distributions.normal import Normal
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import pdb

def NNtrain(model, arg, champDataset, val_x, val_y):
    writer = SummaryWriter(log_dir=arg.log_path)
    train_loader = DataLoader(dataset=champDataset,
                        batch_size=arg.batch_size,
                        shuffle=True,
                        num_workers=2)

    optimizer = optim.Adam(model.parameters(), lr=arg.lr, weight_decay=arg.weight_decay)
    criterion = nn.NLLLoss()
    iter_num = 0

    for epoch in range(1, arg.epochs+1):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            feature, target = data
            feature, target = Variable(feature), Variable(target)
            if arg.use_cuda:
                feature, target = feature, target

            optimizer.zero_grad()
            logit = model(feature.float())
            logit = nn.LogSoftmax(dim=1)(logit)
            loss = criterion(logit, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            iter_num += 1
            if i % 100 == 99 and i != 0:
                print('[%d, %5d] loss: %.4f' %
                    (epoch, i + 1 , running_loss / 100))
                writer.add_scalar('Loss/train', running_loss / 100, iter_num)
                running_loss = 0.0
        val_acc = eval(val_x, val_y, model, arg)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
    writer.close()

    torch.save(model.state_dict(), arg.model_save_path)

    print('Finished Training')

def DUNNtrain(model, arg, champDataset, val_x, val_y):
    writer = SummaryWriter(log_dir=arg.log_path)
    train_loader = DataLoader(dataset=champDataset,
                        batch_size=arg.batch_size,
                        shuffle=True,
                        num_workers=2)

    optimizer = optim.Adam(model.parameters(), lr=arg.lr, weight_decay=arg.weight_decay)
    criterion = nn.NLLLoss()
    iter_num = 0

    for epoch in range(1, arg.epochs+1):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            feature, target = data
            feature, target = Variable(feature), Variable(target)

            optimizer.zero_grad()
            if arg.embed_type == 'champ2vec':
                mu, sigma = model(feature.long())
            elif arg.embed_type == 'one_hot':
                mu, sigma = model(feature.float())
            target_hat = get_hat(mu, sigma, arg)
            loss = criterion(target_hat, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            iter_num += 1
            if i % 100 == 99 and i != 0:
                print('[%d, %5d] loss: %.4f' %
                    (epoch, i + 1 , running_loss / 100))
                writer.add_scalar('Loss/train', running_loss / 100, iter_num)
                running_loss = 0.0
        val_acc = eval(val_x, val_y, model, arg)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
    writer.close()

    torch.save(model.state_dict(), arg.model_save_path)

    print('Finished Training')

def get_hat(mu, sigma, arg):
    softmax = nn.Softmax()
    normal_dist = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
    y_hat = torch.zeros(mu.shape).float()
    for _ in range(arg.mc_samples):
        normal_samples = normal_dist.sample(sample_shape=mu.shape).squeeze(-1)
        y_sample = mu + sigma.expand(-1,2) * normal_samples
        y_hat += softmax(y_sample)
    y_hat /= arg.mc_samples

    return torch.log(y_hat + torch.Tensor([1e-20]))

def eval(data_x, data_y, model, arg):
    softmax = nn.Softmax(dim=1)
    model.eval()
    with torch.no_grad():
        split_num = 10
        W = torch.tensor([[1.0643,-1.5438],[1.2420,0.7887]])
        b1 = torch.tensor([[0.8721],[1.0325]])
        T = torch.tensor(1.1562)
        pdb.set_trace()
        if arg.uncertainty == 'None':
            predict = model(data_x.float())
        elif arg.uncertainty == 'Data':
            predict, var = model(data_x.float())

        #predict = predict/T #tmp scal
        #predict = (W * (predict.t())).t() #vec scal
        #predict = (torch.mm(W,predict.t())+b1).t() #mat scal

        predict = softmax(predict)
        for i in range(split_num):
            low_prob = 0.5+(0.5/split_num)*i
            high_prob = (0.5+(0.5/split_num)*(i+1))
            index = (predict.max(1).values > low_prob) & (predict.max(1).values < high_prob)
            data_x_unc = data_x[index]
            data_y_unc = data_y[index]
            if arg.uncertainty == 'None':
                pred_unc = model(data_x_unc)
                
                #pred_unc = pred_unc/T #tmp scal
                #pred_unc = (W * (pred_unc.t())).t() #vec scal
                #pred_unc = (torch.mm(W,pred_unc.t())+b1).t() #mat scal
                
                pred_unc = softmax(pred_unc)                
                acc = (pred_unc.max(1).indices == data_y_unc).sum().item()/len(pred_unc)
                print('Now low prob:{}, high prob:{}'.format(low_prob,high_prob))
                print('test data length:{0}, all accuracy:{1:0.5f}, predicted mean:{2:0.5f}'.format(len(pred_unc), acc, pred_unc.max(1).values.mean().item()))

            elif arg.uncertainty == 'Data':
                pred_unc, var_unc = model(data_x_unc)            
                pred_unc = softmax(pred_unc)
                acc = (pred_unc.max(1).indices == data_y_unc).sum().item()/len(var_unc)
                index = sorted(range(data_x_unc.shape[0]), key=var_unc.squeeze(1).tolist().__getitem__)
                data_x_unc = data_x_unc[index]
                data_y_unc = data_y_unc[index]

                print('Now low prob:{}, high prob:{}'.format(low_prob,high_prob))
                print('test data length:{0}, all accuracy:{1:0.5f}, predicted mean:{2:0.5f}'.format(len(var_unc), acc, pred_unc.max(1).values.mean().item()))

    acc = (predict.max(1).indices == data_y).sum().item()/len(data_y)
    nll = nn.NLLLoss()(torch.log(predict),data_y)
    print('unc : {}'.format(arg.uncertainty))
    print('accuracy is {0:0.5f}, nll is {1:0.5f}'.format(acc, nll))
    return acc

'''
def shuffle(data_x,data_y):
    idx = torch.randperm(len(data_x))
    data_x_shuffle = data_x[idx]
    data_y_shuffle = data_y[idx]
    return data_x_shuffle, data_y_shuffle
'''

def opt_matrix(model, val_x,val_y, option):
    
    criterion = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    
    if option == 'mat':
        W = Variable(torch.randn(2,2), requires_grad=True)
        b = Variable(torch.randn(2,1), requires_grad=True)
        optimizer = optim.RMSprop([W, b], lr=0.0005)
    elif option == 'vec':
        W = Variable(torch.randn(2,1), requires_grad=True)
        optimizer = optim.RMSprop([W], lr=0.0005)
    elif option == 'tem':
        T = Variable(torch.tensor(1.0), requires_grad=True)
        optimizer = optim.RMSprop([T], lr=0.0005)
    for epoch in range(2000):
        if option == 'mat':
            logit = torch.t(model(val_x)).data
            logit = torch.mm(W,logit) + b
            logit = logsoftmax(logit.t())
        elif option == 'vec':
            logit = torch.t(model(val_x)).data
            logit = W * (logit)
            logit = logsoftmax(logit.t())
        elif option == 'tem':
            logit = model(val_x).data
            logit = (logit) / T
            logit = logsoftmax(logit)
        loss = criterion(logit,val_y)
        loss.backward()
        #print(loss)
        optimizer.step()
    pdb.set_trace()

In [ ]:
pip install cassiopeia

     |████████████████████████████████| 125 kB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.7 MB/s eta 0:00:011
  Created wheel for cassiopeia: filename=cassiopeia-5.0.1-py3-none-any.whl size=164667 sha256=0a8122ffaec21100b0130b4035412d6e390f77c8c5c1a76f78370e92e1f5ab6f
  Stored in directory: /Users/dongwook/Library/Caches/pip/wheels/0a/64/d1/b6fa5ae1eb7fa08fef41f8aa4bff22bc9d58a978689b71746f
  Created wheel for datapipelines: filename=datapipelines-1.0.7-py3-none-any.whl size=23209 sha256=cbf0f51db20b8164f8bcb81d6af9802ab8269b8b2337a827c8c2dba520ec3daa
  Stored in directory: /Users/dongwook/Library/Caches/pip/wheels/a5/ba/bf/09aed39441b2cccedf5513f97b7874313a731ad6ab9be9c268
Successfully built cassiopeia datapipelines
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cassiopeia as cass
from cassiopeia.data import Queue, Position, Region
from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch
import pandas as pd
from pandas import DataFrame as df
import numpy as np
import random, arrow, csv, time, pdb
import datetime

my_api_key = "RGAPI-93b149cc-20a5-464c-add2-f2f2998611e8"
queueType = Queue.ranked_solo_fives
patch_ver = "11.17"
matchdata_num_of_each_region = 40000
test_sample_per_region = 2500
save_csv_per = 8000

# ID from the match data (which it provides directly).
champion_id_to_name_mapping = {champion.id: champion.name for champion in cass.get_champions(region="NA")}

def save_gosu_player(region : str):
    lol_gosu_names = []
    try:
        challenger_league = cass.get_challenger_league(queue=queueType, region=region)
        challenger_entries = challenger_league.entries

        for challenger in challenger_entries:
            lol_gosu_names.append(challenger.summoner.account_id)
        
        grandmaster_league = cass.get_grandmaster_league(queue=queueType, region=region)
        grandmaster_entries = grandmaster_league.entries

        for grandmaster in grandmaster_entries:
            lol_gosu_names.append(grandmaster.summoner.account_id)

        master_league = cass.get_master_league(queue=queueType, region=region)
        master_entries = master_league.entries
        for master in master_entries:
            lol_gosu_names.append(master.summoner.account_id)
    except:
        print("Server error raise. Wait for 1 second.")
        time.sleep(1)
        pass

    return lol_gosu_names

def collect_gosu_ids(region_str):
    worlds_gosu_players_id = {}
    
    try:
        with open("./LOLData/gosuIDs"+patch_ver+".csv","r") as csvfile:
            csvreader = csv.reader(csvfile)
            for region_name, gosu_ids in  zip(region_str, csvreader):
                worlds_gosu_players_id[region_name] = gosu_ids
            
    except:
        for region_name in region_str:
            gosu_players = save_gosu_player(region_name)
            worlds_gosu_players_id[region_name] = gosu_players
        
        with open("./LOLData/gosuIDs"+patch_ver+".csv","w", newline="") as csvfile:
            for gosu_ids in worlds_gosu_players_id.values():
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(gosu_ids)

    return worlds_gosu_players_id

def filter_match_history(summoner, patch):
    end_time = patch.end
    if end_time is None:
        end_time = arrow.now()
    match_history = MatchHistory(summoner=summoner, queues={queueType}, begin_time=patch.start, end_time=end_time)
    return match_history


def collect_gosu_matches(players_id):
    match_ids_dict = {}

    try:
        region_str = players_id.keys()
        with open("./LOLData/matchIDs"+patch_ver+".csv","r") as csvfile:
            csvreader = csv.reader(csvfile)
            for region_name, gosu_ids in  zip(region_str, csvreader):
                match_ids_dict[region_name] = gosu_ids

    except:
        for region_name in players_id.keys():
            patch = Patch.from_str(patch_ver, region=region_name)

            summoner_ids = players_id[region_name]

        
            match_ids = set([])
            for i, summoner_id in enumerate(summoner_ids):
                try:
                    new_summoner = Summoner(account_id=summoner_id, region=region_name)
                    matches = filter_match_history(new_summoner, patch)
                    match_ids.update([match.id for match in matches])
                    print('Now match ids length is {}'.format(len(match_ids)))
                    print('Now used summoners are {} / {}'.format(i+1,len(summoner_ids)))

                except:
                    print("Server error raise. Wait for 1 second.")
                    time.sleep(1)
                    pass

            match_ids = list(match_ids)
            match_ids_dict[region_name] = match_ids
            with open("./LOLData/matchIDs"+patch_ver+".csv","a", newline="") as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerow(match_ids)
           
    return match_ids_dict

def collect_champ_comp(matches_id, seperation):
    #match_data={'blue_win':[],'blue_top_lane':[],'blue_jungle':[],'blue_mid_lane':[],'blue_bot_lane':[],'blue_bot_lane1':[],'blue_bot_lane2':[],
                #'red_top_lane':[],'red_jungle':[],'red_mid_lane':[],'red_bot_lane':[],'red_bot_lane1':[],'red_bot_lane2':[]}
    match_data={'blue_win':[],'blue_1':[],'blue_2':[],'blue_3':[],'blue_4':[],'blue_5':[],
                'red_1':[],'red_2':[],'red_3':[],'red_4':[],'red_5':[],'blue_death_diff':[],'red_death_diff':[], 'gold_diff':[], 'xp_diff':[], 'timeline':[]}

    for region_name in matches_id.keys():
        match_ids = matches_id[region_name]

        for k, match_id in enumerate(match_ids):
            try:
                match_id = int(match_id)
                new_match = Match(id=match_id, region=region_name)
            
                #if new_match.blue_team.participants[0].lane != None:
                frame_len = len(new_match.timeline.frames)
                match_data['timeline'].extend(list(range(frame_len)))
                
                blue_gold = [0 for col in range(frame_len)]
                red_gold = [0 for col in range(frame_len)]
                blue_xp = [0 for col in range(frame_len)]
                red_xp = [0 for col in range(frame_len)]
                blue_death = [0 for col in range(frame_len)]
                red_death = [0 for col in range(frame_len)]

                if new_match.blue_team.win:
                    for _ in range(frame_len):
                        match_data['blue_win'].append(1)
                else:
                    for _ in range(frame_len):
                        match_data['blue_win'].append(0)
                for i, blue_participant in enumerate(new_match.blue_team.participants):
                    for j, frame in enumerate(blue_participant.timeline.frames):
                        match_data['blue_'+str(i+1)].append(champion_id_to_name_mapping[blue_participant.champion.id])
                        blue_gold[j] += frame.gold_earned
                        blue_xp[j] += frame.experience
                        blue_death[j] += blue_participant.cumulative_timeline[datetime.timedelta(minutes=j)].deaths                        
                
                for i, red_participant in enumerate(new_match.red_team.participants):
                    for j, frame in enumerate(red_participant.timeline.frames):
                        match_data['red_'+str(i+1)].append(champion_id_to_name_mapping[red_participant.champion.id])
                        red_gold[j] += frame.gold_earned
                        red_xp[j] += frame.experience
                        red_death[j] += red_participant.cumulative_timeline[datetime.timedelta(minutes=j)].deaths
                
                for i in range(len(blue_death)):
                    if i == 0:
                        match_data['blue_death_diff'].append(0)
                    else:
                        match_data['blue_death_diff'].append(blue_death[i] - blue_death [i-1])
                for i in range(len(red_death)):
                    if i == 0:
                        match_data['red_death_diff'].append(0)
                    else:
                        match_data['red_death_diff'].append(red_death[i] - red_death [i-1])
                
                match_data['gold_diff'].extend((np.array(blue_gold) - np.array(red_gold)).tolist())
                match_data['xp_diff'].extend((np.array(blue_xp) - np.array(red_xp)).tolist())
                                
            except:
                print("Server error raise. Wait for 2 second.")
                time.sleep(2)
                pass
            len_list=[len(length) for length in match_data.values()]
            min_len = min(len_list)
            for data_list in match_data.values():
                if min_len < len(data_list):
                    for _ in range(frame_len):
                        del data_list[-1]

            print('Now match ID length is {} / {}'.format(k, len(match_ids)))
            print('Now match Data length is', len_list)

            if (k + 1) % save_csv_per == 0 or (k + 1) == len(match_ids):
                match_data_df = df(match_data)
                match_data_df.to_csv("./LOLData/MatchData"+patch_ver+seperation+".csv",mode='a')
                match_data.clear()
                #pdb.set_trace()
                match_data={'blue_win':[],'blue_1':[],'blue_2':[],'blue_3':[],'blue_4':[],'blue_5':[],
                'red_1':[],'red_2':[],'red_3':[],'red_4':[],'red_5':[],'blue_death_diff':[],'red_death_diff':[], 'gold_diff':[], 'xp_diff':[], 'timeline':[]}
    match_data.clear()

if __name__ == "__main__":
    cass.set_riot_api_key(my_api_key)
    '''
    region_str = []
    
    for i in Region:
        region_str.append(i.value)
    '''
    region_str = ["KR","EUW","NA","EUNE"]
    worlds_gosu_players_id_dict = collect_gosu_ids(region_str)

    match_ids_dict = collect_gosu_matches(worlds_gosu_players_id_dict)

    train_match_ids_dict = {}
    test_match_ids_dict = {}
    for region in region_str:
        test_match_ids_dict[region] = random.sample(match_ids_dict[region], test_sample_per_region)
        train_match_ids_dict[region] = list(set(match_ids_dict[region])-set(test_match_ids_dict[region]))
    del match_ids_dict

    collect_champ_comp(train_match_ids_dict, 'train')
    collect_champ_comp(test_match_ids_dict, 'test')



Making call: https://kr.api.riotgames.com/lol/summoner/v4/summoners/VXHgHrnmuyqRQLnkrenG3EGVBp7bwr3wQW954n9D80WBr0Q
Server error raise. Wait for 1 second.
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/RIUkPYRlm5EgVyzUFr9sPKvoSEj3AcJ8a88eC2c0mGsfpuE
Server error raise. Wait for 1 second.
Making call: https://na1.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/YzfsSBKXpJ2tyrEZjBzDehtVN0z4yoLgsch-mRuCRT6TifuOtb1I9J7v9A
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/PwaSJm_SHeZjeMI38nUg4L9tP8Fj574EbncWg3BsM0JbM-0
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/hbYp3JHqAzxCAidlsRzVE_9O7x7QeRBatD3FoDPDMDfx1Uk
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/iqK17vCwQHH9JTMYCd4J2rKtpv61Ud47GdZ6WwwJ9jpx2kUP
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/fI0EGFKOVxBXwi0M5e9XYpntK4JFciOD_Uaut61

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/RoI1-sZzNktts9m40pBwgM_drF1j57pPKfVgkj9mpXhE9JI
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/CGQalY0OYuWj_oaBWa_p1o2cewBOr6ZYTKbbZPJ4RFpizf8
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/k4R5WQ3ZfqdNaSHVY_doELLb2DIHVKN5HyEIr0mBWM0_Em4
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/Sfp76C-Iz8B3m8T4E9-nfZMClDXmD6U-4Qnz9HS_RbXEhxw
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/oynuYgKBkR0eW0NnXWyjjjFbRMD4Sau0FaY90MM5MZpBOkBXyWeR-FgAow
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/go92EXb2L7v79tmKBRWm5nl4fq6hh8wgSGCn0sj1ShMpG_w
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/HSUz6rfMiUNtFQbKbPPxtuUYIxaRrdznnKdZjqFODlm7WBU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/fzfEzu9EzhUv15sNho1AksRo0d3uYcDbYtVheBbc9DI0eLk
Making call: https://na1.api.riotgames.com/lol/summon

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/TeLo1038mAtBGdf1j2UMHQJS5kuV3L_NF2x5x-TasPHAeYw2ffQS_DpFkw
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/bBsF5xrF11VbzeYmfpy67GNm25zgiXIxIgUIuOdgjCCCd1QD10xyTNnFDg
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/GRk3_cgrgRTM_Y9uWwhT4jxfqdpTrMcVBptdv9aEwniX2MXtvzQE_xt5SA
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/INENadsXzzKWlDy6PWKxu0Wl23OE1WU3yQ1mBZ760QO5igjO
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/SYOnZiUV0yfqPl-zkEg5dsNTOnljszHAyyVlWld3nos0miA
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/nVIdDGoMMs00cB6_rke0i6wHCnYUVgyaC0d0KBdU3p_gdqrQr9OIFScMtQ
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/1ooNJWjuZ_FUIea7Wm8esXiB0TrXaQMhaDiVcfYT8VBlJe4
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/RqE-1z6tpxyGAX7Nl78lFFpdnbenYRhSDtKk3_oiKB0fHc0
Making call: https:

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/1cBUKEhvzVb1tn83icKyU-0Y9QB1qkGk381akLJuHadoK6N136sknz_IEQ
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/IFdXc9v4pu_uwuu73n-VnF-yF9l1xDVgWjYnHmeGIBOIfX4
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/UK3scbBYa3itQ4Q25A9wAVgoQ0CYJeMYabfQMIaqLr3topE
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/Cg89-eSvl5D1os_rL7lGD4AWbFHVOrgeFL01QilyNuUdVEw
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/nDHDbGwFWlDxiYGlTSPomB855YVbY7gCaBJqSTd6QKTsUOQ
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/5ipzcq5gJPlZpf1vfcYjke-gC16_vX1czn8_hOUfV5NWcDM
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/64lRxyihNz36f7fIA35jUE7gNlUJZpUCU1zMDDZF0-iGwhSnO_OGBiybtw
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/PTX1ThQcJ-JALzcmLhokOJjyOcCr4b7gNs2K_ikMSGFJF48
Making call: https://na1.api.riotgames.com

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/0mGPguHDXfAZlqP0jzvMuk0JMOw9tFTdiRr3mZDpdudX3XM
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/5yHQMfjAXRDVFQbS2QIbmKY0Eocgd0leszo0BVA7uK-PTtE
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/pIeJJIBUD4gq89CICpCTwdnGtPx2woEGYXCw613oAYRmcmc
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/Psm8DTjfI3bortvtyvQARzJmElE6j84kvV9voYNTUeCPFZ4u
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/QT6bE2sEiOCRyJ8VPS-nLjpP4yslCqJ3N_wTlSoh8irGIg0
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/dPVVcs2-IV_9kA16BDRmxE6M89VruQvBqyn6YNJ1_I6GCMs
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/pA_LsJKCfZ3S9BUwbbVdXbamkKFm7qpi3wY_Tsjm1ks90LI
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/yWgV9Zj0R7bi57MYkTxXa_Q0eJdaRjJx_Q8W4uwrDYwvuYU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summ

...

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/fjJZcJxqNEJxQxSUwLZA1C9vwoDUJLZ41YlhW-lDSl8E1zc
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/t2eM5VNC9qaa76xtb7if4uW5xZ42SMqkj5lbSEXIhh9HaJA
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/1z8nogvM_4RVo9Qz4CxyZ6U2PcCVcU-ApZgCBPM4tVMEbVk
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/6SlXu6yvko_8tvHxGj_Wm_SihEnl5V5uPBuosWkWXnMKpjk
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/ZJ0Kv1aFy8lWpEnFc4ZMQbGu5G1J1-AXOIyfu51CezE5-OY
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/JisDmxhXTSBhvh0HrjkxsUfWiWnnHFZWCxMy4QpwwXJpgZc
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/u1J1xhKYYhB97eVKyk2RJVjk5S2MQVBgrgXsh9le3jxa6qM
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/6l5HxtiAL1_0TbygZVYmm7CqhpSVwH7_HweVN98YbxtqtKkNTn9ojquhyg
Making call: https://na1.api.riotgames.com/lol/summon

Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/vUy8IO8OKVszYCNKTkpvA3YfJMWlqUN-Wz1mKy5_sHLSc8w
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/Fe4snJC9wTXP7fxB7ZEw15SqqEXts389eraa95vyI5A
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/gf-sOMT2wEsEaBYcPpPogyxWpEPjhT6n3OWd_2H874Ab3cwt4GLMKkkCmQ
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/im8jGQ7ooFa_XXbmN9_n6eWQ-hMdIomY10rHU-1a-GFnmSU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/mwldDX5P3XNoaO03qxWWbNvg3D05Qqg6IqGwB40TN4AVwlo3
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/Zk62mw04q0L6uqeuf9A7ciYbG5PbrQ3eW3HsmdUkLyXMu2U
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/kkL30BHEEN8BiFn02pnh0epoIhwjmFmtkz9xY2MbbsZ22bE
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/4ZNCJucumaXa6NwzFLF8MAv0BFX2Mi06H51dVBN1XblCrYc
Making call: https://na1.api.riotgames.com/lol/summoner/

Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/ITXdLDFZkOZWjX62dl3iArN8FqlQcrcV0_hy-BmAiNnmBb4
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/0rdU3wRye2LqKg8MttC4CpMjxqRiMgIxlYuQL8sWnucDZlM
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/ytTdJ5ZaWk3HF1WUikI6yz3liPZkI8UtWeKulIUP0qMHmAc
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/6MzDUQiFL-hMVD_yOS137R8NALu1gOS5jbo7MlNOubtJnGI
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/1H9WCzyPNzSHb6Cqx43LVwhOu7FHyD3rMUpZtAZod2z5cN8
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/ND1ONP8ZowJm8aj7t1wkkdVwsZji4m7fy5AL_u_cUtEeHCM
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/9SGrdCbTz_P4nv-d5BRGmDKNZBiZnDoIrF2lApTnDzccO20
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/3Bast-yWbff_HFDDk_AFGD_L6sLEMueiKMLXe6J6pdiLWfI
Making call: https://eun1.api.riotgames.com/lol/summoner

Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/WAHWSL9OWB3_YhHs0mbcSORM8lCh8HHs4RETEWBoi7OVKjQ
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/b7fFUrl-ms_Zlj1colLTxsDUOTdzy04dlhLMIkaUDGAZpRA
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/O-JF-u3o1N_jOchoq0_4zfhyCnj2s5rB5FlJOp14xazBhKo
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/Le8AKqiXhIeEFis4NbYa1nSqNiwiPj15E2DBLj0hSyOKOr0
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/TN1UVPTAkYLp044Kzft7M-kPwaFBGEY5kSnTe9gsjNp2xZo
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/F3kumaLEZ4TUJmi4TR8E9V7IbpFr1D2-nTbkHytqy6-E8HE
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/eX7j-8dip5XgRp99Sxs2zIe3du8csbytEBE372EHwG8m2Uo
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/YNHFQq3yI1iHm4hdlMBriq-G8j5YHJ_nmu4c2rnwlNSUGFI
Making call: https://eun1.api.riotgames.com/lol/summoner

Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/EDDv0bFZgiIBR4AGFJ7l6BjBC5V7N-nkSk5W4h9LlIE32nU
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/XDsAWVpnu3dbhhjrbYDUkv1DFEOjvLjFBm_Q8ZNp2MBncc8
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/clKss9hh_b_6YrOtgaCCNprJmkn-OG1Zzv7Cz7wy-SKXmV4
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/kA4r-9mYg6DXKc-djQaTCpc5uzfDgACiDwABUErQ8RUBT0k
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/CzNleGxBWh6IEbZGUj7ZZQ2PDPQO_ti3AF-lr9e2VgUA-lw
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/0S1ZCjwkosxVnD1_iY5Pc4qPDj5p_kqfjG71m2oW_ca1s8c
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/rIq7sPa88rsDO-87XUNw9JzN8kmLnQlNWiJ2s18M7up23VA
Making call: https://eun1.api.riotgames.com/lol/summoner/v4/summoners/BXka9TrHYosS7jj230OITZ0mt9nuIeGlx2JFlyOx013EMtg
Making call: https://eun1.api.riotgames.com/lol/summoner

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from preprocess import * #Preprocess, ChampDataset
from model import * #CNNModel, MUCNNModel, RNNModel, MURNNModel, OHModel, MUOHModel
from arguments import get_args
import train
from datetime import datetime
now = datetime.now()
import os
import pdb

preprocess = Preprocess()
arg = get_args()
'''
def oneHotEncodding(labels):
    onehot_encoded = list()
    for value in labels:
        target = [0 for _ in range(2)]
        target[value] = 1
        onehot_encoded.append(target)
    return onehot_encoded
'''
def main():

    allChamp, matchComp, blueWin = preprocess.lolDataSet(arg, "train")
    _, test_x, test_y = preprocess.lolDataSet(arg, "test")

    test_x = torch.tensor(test_x).float()
    test_y = torch.tensor(test_y.reshape(-1)).long()

    val_x = torch.tensor(matchComp[0:arg.val_set_size]).float()
    val_y = torch.tensor(blueWin[0:arg.val_set_size].reshape(-1)).long()
    train_x = matchComp[arg.val_set_size:-1]
    train_y = blueWin[arg.val_set_size:-1].reshape(-1)

    champDataset = ChampDataset(train_x,train_y)

    arg.champ_num = len(allChamp)

    if arg.in_game:
        arg.log_path = arg.log_path + 'in-game/'
        arg.model_save_path = arg.model_save_path + 'in-game/'
    else:
        arg.log_path = arg.log_path + 'pregame/'
        arg.model_save_path = arg.model_save_path + 'pregame/'

    arg.log_path = arg.log_path + arg.embed_type + '/' + arg.uncertainty + '/' + now.strftime("%Y%m%d-%H%M%S")
    arg.model_save_path = arg.model_save_path + arg.embed_type + '/' + arg.uncertainty
    if not(os.path.isdir(arg.model_save_path)):
        os.makedirs(os.path.join(arg.model_save_path))
    if arg.saved_model_time == None:
        arg.model_save_path = arg.model_save_path + '/' + now.strftime("%Y%m%d-%H%M%S") + '.pt'

    if arg.embed_type == 'one_hot' and arg.uncertainty == 'None':
        model = OHModel(arg)

        if arg.saved_model_time == None:
            train.NNtrain(model, arg, champDataset, val_x, val_y)
        else:
            arg.model_save_path = arg.model_save_path + '/' +arg.saved_model_time + '.pt'
            model.load_state_dict(torch.load(arg.model_save_path))
            option = 'mat' #vec, mat, tem
            train.opt_matrix(model, val_x,val_y, option)

        train.eval(test_x,test_y,model,arg)


    elif arg.embed_type == 'one_hot' and arg.uncertainty == 'Data':
        model = DUOHModel(arg)

        if arg.saved_model_time == None:
            train.DUNNtrain(model, arg, champDataset, val_x, val_y)
        else:
            arg.model_save_path = arg.model_save_path + '/' +arg.saved_model_time + '.pt'
            model.load_state_dict(torch.load(arg.model_save_path))
        
        train.eval(test_x,test_y,model,arg)


if __name__ == '__main__':
    main()

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.distributions.normal import Normal
from torch.autograd import Variable
import torch.optim as optim
import torch.nn as nn
import pdb

def NNtrain(model, arg, champDataset, val_x, val_y):
    writer = SummaryWriter(log_dir=arg.log_path)
    train_loader = DataLoader(dataset=champDataset,
                        batch_size=arg.batch_size,
                        shuffle=True,
                        num_workers=2)

    optimizer = optim.Adam(model.parameters(), lr=arg.lr, weight_decay=arg.weight_decay)
    criterion = nn.NLLLoss()
    iter_num = 0

    for epoch in range(1, arg.epochs+1):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            feature, target = data
            feature, target = Variable(feature), Variable(target)
            if arg.use_cuda:
                feature, target = feature, target

            optimizer.zero_grad()
            logit = model(feature.float())
            logit = nn.LogSoftmax(dim=1)(logit)
            loss = criterion(logit, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            iter_num += 1
            if i % 100 == 99 and i != 0:
                print('[%d, %5d] loss: %.4f' %
                    (epoch, i + 1 , running_loss / 100))
                writer.add_scalar('Loss/train', running_loss / 100, iter_num)
                running_loss = 0.0
        val_acc = eval(val_x, val_y, model, arg)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
    writer.close()

    torch.save(model.state_dict(), arg.model_save_path)

    print('Finished Training')

def DUNNtrain(model, arg, champDataset, val_x, val_y):
    writer = SummaryWriter(log_dir=arg.log_path)
    train_loader = DataLoader(dataset=champDataset,
                        batch_size=arg.batch_size,
                        shuffle=True,
                        num_workers=2)

    optimizer = optim.Adam(model.parameters(), lr=arg.lr, weight_decay=arg.weight_decay)
    criterion = nn.NLLLoss()
    iter_num = 0

    for epoch in range(1, arg.epochs+1):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(train_loader):
            feature, target = data
            feature, target = Variable(feature), Variable(target)

            optimizer.zero_grad()
            if arg.embed_type == 'champ2vec':
                mu, sigma = model(feature.long())
            elif arg.embed_type == 'one_hot':
                mu, sigma = model(feature.float())
            target_hat = get_hat(mu, sigma, arg)
            loss = criterion(target_hat, target)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            iter_num += 1
            if i % 100 == 99 and i != 0:
                print('[%d, %5d] loss: %.4f' %
                    (epoch, i + 1 , running_loss / 100))
                writer.add_scalar('Loss/train', running_loss / 100, iter_num)
                running_loss = 0.0
        val_acc = eval(val_x, val_y, model, arg)
        writer.add_scalar('Accuracy/val', val_acc, epoch)
    writer.close()

    torch.save(model.state_dict(), arg.model_save_path)

    print('Finished Training')

def get_hat(mu, sigma, arg):
    softmax = nn.Softmax()
    normal_dist = Normal(torch.tensor([0.0]), torch.tensor([1.0]))
    y_hat = torch.zeros(mu.shape).float()
    for _ in range(arg.mc_samples):
        normal_samples = normal_dist.sample(sample_shape=mu.shape).squeeze(-1)
        y_sample = mu + sigma.expand(-1,2) * normal_samples
        y_hat += softmax(y_sample)
    y_hat /= arg.mc_samples

    return torch.log(y_hat + torch.Tensor([1e-20]))

def eval(data_x, data_y, model, arg):
    softmax = nn.Softmax(dim=1)
    model.eval()
    with torch.no_grad():
        split_num = 10
        W = torch.tensor([[1.0643,-1.5438],[1.2420,0.7887]])
        b1 = torch.tensor([[0.8721],[1.0325]])
        T = torch.tensor(1.1562)
        pdb.set_trace()
        if arg.uncertainty == 'None':
            predict = model(data_x.float())
        elif arg.uncertainty == 'Data':
            predict, var = model(data_x.float())

        #predict = predict/T #tmp scal
        #predict = (W * (predict.t())).t() #vec scal
        #predict = (torch.mm(W,predict.t())+b1).t() #mat scal

        predict = softmax(predict)
        for i in range(split_num):
            low_prob = 0.5+(0.5/split_num)*i
            high_prob = (0.5+(0.5/split_num)*(i+1))
            index = (predict.max(1).values > low_prob) & (predict.max(1).values < high_prob)
            data_x_unc = data_x[index]
            data_y_unc = data_y[index]
            if arg.uncertainty == 'None':
                pred_unc = model(data_x_unc)
                
                #pred_unc = pred_unc/T #tmp scal
                #pred_unc = (W * (pred_unc.t())).t() #vec scal
                #pred_unc = (torch.mm(W,pred_unc.t())+b1).t() #mat scal
                
                pred_unc = softmax(pred_unc)                
                acc = (pred_unc.max(1).indices == data_y_unc).sum().item()/len(pred_unc)
                print('Now low prob:{}, high prob:{}'.format(low_prob,high_prob))
                print('test data length:{0}, all accuracy:{1:0.5f}, predicted mean:{2:0.5f}'.format(len(pred_unc), acc, pred_unc.max(1).values.mean().item()))

            elif arg.uncertainty == 'Data':
                pred_unc, var_unc = model(data_x_unc)            
                pred_unc = softmax(pred_unc)
                acc = (pred_unc.max(1).indices == data_y_unc).sum().item()/len(var_unc)
                index = sorted(range(data_x_unc.shape[0]), key=var_unc.squeeze(1).tolist().__getitem__)
                data_x_unc = data_x_unc[index]
                data_y_unc = data_y_unc[index]

                print('Now low prob:{}, high prob:{}'.format(low_prob,high_prob))
                print('test data length:{0}, all accuracy:{1:0.5f}, predicted mean:{2:0.5f}'.format(len(var_unc), acc, pred_unc.max(1).values.mean().item()))

    acc = (predict.max(1).indices == data_y).sum().item()/len(data_y)
    nll = nn.NLLLoss()(torch.log(predict),data_y)
    print('unc : {}'.format(arg.uncertainty))
    print('accuracy is {0:0.5f}, nll is {1:0.5f}'.format(acc, nll))
    return acc

'''
def shuffle(data_x,data_y):
    idx = torch.randperm(len(data_x))
    data_x_shuffle = data_x[idx]
    data_y_shuffle = data_y[idx]
    return data_x_shuffle, data_y_shuffle
'''

def opt_matrix(model, val_x,val_y, option):
    
    criterion = nn.NLLLoss()
    logsoftmax = nn.LogSoftmax(dim=1)
    
    if option == 'mat':
        W = Variable(torch.randn(2,2), requires_grad=True)
        b = Variable(torch.randn(2,1), requires_grad=True)
        optimizer = optim.RMSprop([W, b], lr=0.0005)
    elif option == 'vec':
        W = Variable(torch.randn(2,1), requires_grad=True)
        optimizer = optim.RMSprop([W], lr=0.0005)
    elif option == 'tem':
        T = Variable(torch.tensor(1.0), requires_grad=True)
        optimizer = optim.RMSprop([T], lr=0.0005)
    for epoch in range(2000):
        if option == 'mat':
            logit = torch.t(model(val_x)).data
            logit = torch.mm(W,logit) + b
            logit = logsoftmax(logit.t())
        elif option == 'vec':
            logit = torch.t(model(val_x)).data
            logit = W * (logit)
            logit = logsoftmax(logit.t())
        elif option == 'tem':
            logit = model(val_x).data
            logit = (logit) / T
            logit = logsoftmax(logit)
        loss = criterion(logit,val_y)
        loss.backward()
        #print(loss)
        optimizer.step()
    pdb.set_trace()